# Bonus Part

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
import time
import matplotlib.pyplot as plt
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#loading and splitting the data

# Image transformations
transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])

# Loading the dataset and assigning the tagets to each class
dataset = torchvision.datasets.ImageFolder(root='/content/dset',transform=transform)

#splitting the data into train data of size 18000, and validation data and testing data of size 6000 each
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,[18000,6000,6000])

# loading the data to loader using the dataloader of torch
traindata_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
validationdata_loader = torch.utils.data.DataLoader(val_dataset,batch_size=64,shuffle=True)
testdata_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=True)

# Printing the targets assignined to each class of data
print(dataset.class_to_idx)

{'dogs': 0, 'food': 1, 'vehicles': 2}


##### VGG13 model has 13 convolution layers embedded in it

In [5]:
class VGG13(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super(VGG13, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2))
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.l1 = nn.Sequential(nn.Linear(512*7*7, 4096),nn.ReLU(),nn.Dropout(0.5))
        self.l2 = nn.Sequential(nn.Linear(4096,4096),nn.ReLU(),nn.Dropout(0.5))
        self.l3 = nn.Sequential(nn.Linear(4096,3)) 
    def forward(self, x):
        output = self.features(output)
        output = self.avgpool(output)
        output = torch.flatten(output,1)
        output = self.l1(output)
        output = self.l2(output)
        output = self.l3(output)
        return x

In [10]:
#Loading the Alexnet to our model and using the crossEntrophyLoass() to calculate the loss in each step in epoch and SGD optimizer to update the weights

classes =3
epochs =5
learning_rate = 0.001

model = VGG13(classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer=torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  
# Train the model


In [12]:
#training the model
train_accuracies = []
test_accuracies = []
start_time= time.time()
for epoch in range(epochs):
    for i, (images, labels) in enumerate(traindata_loader):  
    # loading the tensors to the device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass the model
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize to update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
  # validating the model on the validation set images for each epoch

    correct = 0
    total = 0
    for images,labels in validationdata_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        dummydata, predicted = torch.max(outputs.data, 1)
        total = total+labels.size(0)
        correct = correct+(predicted == labels).sum().item()
        del images, labels, outputs
    correct1 = 0
    total1 = 0

#testing the model for the test accuracy
correct1 = 0
total1 = 0
for images, labels in testdata_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    dummydata, predicted = torch.max(outputs.data, 1)
    total1 += labels.size(0)
    correct1 += (predicted == labels).sum().item()
    del images, labels, outputs
print('final model accuracy on the {} testing images: {} %'.format(total1,100*correct1/total1)) 
print("training time: {} min".format(round(end_time-start_time,2)/60))


final model accuracy on the 6000 testing images: 42.124989 %
training time: 263.87313 min
